# 0. Install and Import Dependencies

In [2]:
import tensorflow as tf
import tensorflow_hub as hub
import cv2
from matplotlib import pyplot as plt
import numpy as np

In [2]:
# Optional if you are using a GPU
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

# 1. Load Model

In [2]:
model = hub.load('https://tfhub.dev/google/movenet/multipose/lightning/1')
movenet = model.signatures['serving_default']

2022-11-16 14:18:18.255751: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2022-11-16 14:18:18.255986: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


# 2. Functions

In [3]:
# Function to loop through each person detected and render
def loop_through_people(frame, keypoints_with_scores, edges, confidence_threshold):
    for person in keypoints_with_scores:
        draw_connections(frame, person, edges, confidence_threshold)
        draw_keypoints(frame, person, confidence_threshold)

In [4]:
def draw_keypoints(frame, keypoints, confidence_threshold):
    y, x, c = frame.shape
    shaped = np.squeeze(np.multiply(keypoints, [y,x,1]))
    
    for kp in shaped:
        ky, kx, kp_conf = kp
        if kp_conf > confidence_threshold:
            cv2.circle(frame, (int(kx), int(ky)), 6, (0,255,0), -1)

In [5]:
EDGES = {
    (0, 1): 'm',
    (0, 2): 'c',
    (1, 3): 'm',
    (2, 4): 'c',
    (0, 5): 'm',
    (0, 6): 'c',
    (5, 7): 'm',
    (7, 9): 'm',
    (6, 8): 'c',
    (8, 10): 'c',
    (5, 6): 'y',
    (5, 11): 'm',
    (6, 12): 'c',
    (11, 12): 'y',
    (11, 13): 'm',
    (13, 15): 'm',
    (12, 14): 'c',
    (14, 16): 'c'
}

In [6]:
def draw_connections(frame, keypoints, edges, confidence_threshold):
    y, x, c = frame.shape
    shaped = np.squeeze(np.multiply(keypoints, [y,x,1]))
    
    for edge, color in edges.items():
        p1, p2 = edge
        y1, x1, c1 = shaped[p1]
        y2, x2, c2 = shaped[p2]
        
        if (c1 > confidence_threshold) & (c2 > confidence_threshold):      
            cv2.line(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0,0,255), 4)

In [7]:
cap = cv2.VideoCapture(1)

while cap.isOpened():
    ret, frame = cap.read()
    
    # Recolor Feed
    #image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    #image.flags.writeable = False
    
    # Recolor image back to BGR for rendering
    #image.flags.writeable = True
    #image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    #img = image.copy()
    #img = tf.image.resize_with_pad(tf.expand_dims(img, axis=0), 384,640)
    #input_img = tf.cast(img, dtype=tf.int32)
    
    # Resize image
    img = frame.copy()
    img = tf.image.resize_with_pad(tf.expand_dims(img, axis=0), 384,640)
    input_img = tf.cast(img, dtype=tf.int32)
    
    # Detection section
    results = movenet(input_img)
    keypoints_with_scores = results['output_0'].numpy()[:,:,:51].reshape((6,17,3))
    
    # Render keypoints
    loop_through_people(frame, keypoints_with_scores, EDGES, 0.1)
    cv2.imshow('Raw Webcam Feed', frame)
    
    if cv2.waitKey(10) & 0xFF == ord('q'):
        break
        
cap.release()
cv2.destroyAllWindows()

2022-11-16 14:21:50.302606: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)


KeyboardInterrupt: 

In [7]:
keypoints_with_scores

NameError: name 'keypoints_with_scores' is not defined

In [14]:
keypoints_with_scores = keypoints_with_scores[:4]
practice = np.delete(keypoints_with_scores, 2, 2)

In [15]:
practice

array([[[0.4896776 , 0.5958009 ],
        [0.41025823, 0.6354448 ],
        [0.4078443 , 0.5540599 ],
        [0.4527523 , 0.6707106 ],
        [0.4648654 , 0.4935092 ],
        [0.5815653 , 0.6894378 ],
        [0.5959259 , 0.47572374],
        [0.54901516, 0.6995781 ],
        [0.5400668 , 0.4851966 ],
        [0.54093736, 0.6715024 ],
        [0.53551745, 0.5101397 ],
        [0.54855776, 0.6792631 ],
        [0.52256227, 0.58890367],
        [0.5098033 , 0.657408  ],
        [0.5308982 , 0.54115605],
        [0.37397385, 0.66054004],
        [0.3648909 , 0.6054861 ]],

       [[0.4896776 , 0.5958009 ],
        [0.41025823, 0.6354448 ],
        [0.4078443 , 0.5540599 ],
        [0.4572904 , 0.6697147 ],
        [0.46943355, 0.49309072],
        [0.80196375, 0.7454193 ],
        [0.76395077, 0.41590744],
        [0.93161744, 0.8046957 ],
        [0.9384875 , 0.412608  ],
        [0.944553  , 0.7196078 ],
        [0.89325804, 0.48518163],
        [0.9332437 , 0.6946528 ],
        [0.9

In [26]:
finalPractice = np.delete(practice, [0,1,2,3,4], 1)

In [27]:
finalPractice

array([[[0.5815653 , 0.6894378 ],
        [0.5959259 , 0.47572374],
        [0.54901516, 0.6995781 ],
        [0.5400668 , 0.4851966 ],
        [0.54093736, 0.6715024 ],
        [0.53551745, 0.5101397 ],
        [0.54855776, 0.6792631 ],
        [0.52256227, 0.58890367],
        [0.5098033 , 0.657408  ],
        [0.5308982 , 0.54115605],
        [0.37397385, 0.66054004],
        [0.3648909 , 0.6054861 ]],

       [[0.80196375, 0.7454193 ],
        [0.76395077, 0.41590744],
        [0.93161744, 0.8046957 ],
        [0.9384875 , 0.412608  ],
        [0.944553  , 0.7196078 ],
        [0.89325804, 0.48518163],
        [0.9332437 , 0.6946528 ],
        [0.9464004 , 0.5182978 ],
        [0.9179814 , 0.7406748 ],
        [0.9415144 , 0.43064055],
        [0.63000584, 0.61999387],
        [0.6248572 , 0.6092218 ]],

       [[0.7748089 , 0.6401588 ],
        [0.7610394 , 0.3810438 ],
        [0.86696786, 0.616751  ],
        [0.91269886, 0.35524464],
        [0.80453444, 0.51806843],
        [0

In [34]:
finalPractice[0]

array([[0.5815653 , 0.6894378 ],
       [0.5959259 , 0.47572374],
       [0.54901516, 0.6995781 ],
       [0.5400668 , 0.4851966 ],
       [0.54093736, 0.6715024 ],
       [0.53551745, 0.5101397 ],
       [0.54855776, 0.6792631 ],
       [0.52256227, 0.58890367],
       [0.5098033 , 0.657408  ],
       [0.5308982 , 0.54115605],
       [0.37397385, 0.66054004],
       [0.3648909 , 0.6054861 ]], dtype=float32)

In [31]:
vectorList = [
    [0,1],
    [0,2],
    [1,3],
    [2,4],
    [3,5],
    [0,6],
    [1,7],
    [6,7],
    [6,8],
    [7,9],
    [8,10],
    [9,10]
]

In [35]:
for j in vectorList:
    print(i[0])
    print(i[1])
    print("====")

0
1
====
0
2
====
1
3
====
2
4
====
3
5
====
0
6
====
1
7
====
6
7
====
6
8
====
7
9
====
8
10
====
9
10
====


In [50]:
for person in finalPractice:
    for i in vectorList:
#         print(person[i[0]])    # first coordinate (y,x)
#         print(person[i[1]])    # second coordinate (y,x)
        print(person[i[1]] - person[i[0]])    # vector(second - first)
        print("====")

[0.5815653 0.6894378]
[0.5959259  0.47572374]
[ 0.01436061 -0.21371406]
====
[0.5815653 0.6894378]
[0.54901516 0.6995781 ]
[-0.03255016  0.0101403 ]
====
[0.5959259  0.47572374]
[0.5400668 0.4851966]
[-0.05585915  0.00947285]
====
[0.54901516 0.6995781 ]
[0.54093736 0.6715024 ]
[-0.0080778 -0.0280757]
====
[0.5400668 0.4851966]
[0.53551745 0.5101397 ]
[-0.00454932  0.02494311]
====
[0.5815653 0.6894378]
[0.54855776 0.6792631 ]
[-0.03300756 -0.01017469]
====
[0.5959259  0.47572374]
[0.52256227 0.58890367]
[-0.07336366  0.11317992]
====
[0.54855776 0.6792631 ]
[0.52256227 0.58890367]
[-0.02599549 -0.09035945]
====
[0.54855776 0.6792631 ]
[0.5098033 0.657408 ]
[-0.03875446 -0.02185512]
====
[0.52256227 0.58890367]
[0.5308982  0.54115605]
[ 0.00833595 -0.04774761]
====
[0.5098033 0.657408 ]
[0.37397385 0.66054004]
[-0.13582945  0.00313205]
====
[0.5308982  0.54115605]
[0.37397385 0.66054004]
[-0.15692437  0.11938399]
====
[0.80196375 0.7454193 ]
[0.76395077 0.41590744]
[-0.03801298 -0.3295

In [1]:
np.squeeze(vector)

NameError: name 'np' is not defined